In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = [15, 10]

## Read and fix data

In [153]:
dataDir = "../../Data for Tax Equity Project/eia_data/eia8602018/"

In [154]:
utils = pd.read_csv(dataDir + '/utility_2018.csv', header=1)
owners = pd.read_csv(dataDir + 'owner_2018.csv', header=1)

In [155]:
gen_data = pd.read_csv(dataDir + 'gen_2018.csv')
gen_data.drop(['Unnamed: 0'],axis=1,inplace=True)
gen_data = gen_data[~gen_data['Utility ID'].str.contains('NOTE', na=False)]
gen_data['Utility ID'] = pd.to_numeric(gen_data['Utility ID'])

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,16,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [156]:
utils.columns = utils.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
owners.columns =  owners.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
gen_data.columns = gen_data.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
owners.columns =  owners.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

Now I want to merge the utility data with the generator data so I can do some sick analyses bro

I'll start by merging the utility list with the list of generators (all in 2018), so I'll see if any columns will conflict

In [167]:
[print(x) for x in gen_data.columns if x in utils.columns]

utility_id
utility_name


[None, None]

I'm merging by utility_id so let's fix the state variable

In [166]:
gen_data = gen_data.rename(columns={'state': 'plant_state'})

In [168]:
merged = gen_data.merge(utils, how='left', on=['utility_id','utility_name'])

Various checks assures me that the merge was successful. So now I want to merge in ownership data in a clever way

In [136]:
gen_data[gen_data['utility_name'] == 'Tennessee Valley Authority']

,utility_name,plant_code,plant_name,state,county,generator_id,technology,prime_mover,unit_code,ownership,...,planned_energy_source_1,planned_new_nameplate_capacity_mw,planned_repower_month,planned_repower_year,other_planned_modifications?,other_modifications_month,other_modifications_year,multiple_fuels?,cofire_fuels?,switch_between_oil_and_natural_gas?
utility_id,,,,,,,,,,,,,,,,,,,,,
18642,Tennessee Valley Authority,46.0,Browns Ferry,AL,Limestone,1,Nuclear,ST,NaN,S,...,NaN,,,,NaN,,,N,NaN,NaN
18642,Tennessee Valley Authority,46.0,Browns Ferry,AL,Limestone,2,Nuclear,ST,NaN,S,...,NaN,,,,NaN,,,N,NaN,NaN
18642,Tennessee Valley Authority,46.0,Browns Ferry,AL,Limestone,3,Nuclear,ST,NaN,S,...,NaN,,,,NaN,,,N,NaN,NaN
18642,Tennessee Valley Authority,47.0,Colbert,AL,Colbert,GT1,Natural Gas Fired Combustion Turbine,GT,NaN,S,...,NaN,,,,NaN,,,Y,N,Y
18642,Tennessee Valley Authority,47.0,Colbert,AL,Colbert,GT2,Natural Gas Fired Combustion Turbine,GT,NaN,S,...,NaN,,,,NaN,,,Y,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18642,Tennessee Valley Authority,55451.0,Magnolia Power Plant,MS,Benton,STG2,Natural Gas Fired Combined Cycle,CA,MCC2,S,...,NaN,,,,NaN,,,N,NaN,NaN
18642,Tennessee Valley Authority,55451.0,Magnolia Power Plant,MS,Benton,STG3,Natural Gas Fired Combined Cycle,CA,MCC3,S,...,NaN,,,,NaN,,,N,NaN,NaN
18642,Tennessee Valley Authority,55694.0,Quantum Choctaw Power LLC,MS,Choctaw,CT1,Natural Gas Fired Combined Cycle,CT,CC1,S,...,NaN,,,,NaN,,,N,N,N


In [135]:
utils[utils['utility_name']=='Tennessee Valley Authority']

,utility_name,street_address,city,state,zip,owner_of_plants_reported_on_form?,operator_of_plants_reported_on_form?,asset_manager_of_plants_reported_on_form?,other_relationships_with_plants_reported_on_form?,entity_type
Utility ID,,,,,,,,,,
18642,Tennessee Valley Authority,WT 4B-K 400 W Summit Hill Driv,Knoxville,TN,37902,Y,NaN,NaN,NaN,F


In [122]:
utils.columns

Index(['utility_name', 'street_address', 'city', 'state', 'zip',
       'owner_of_plants_reported_on_form?',
       'operator_of_plants_reported_on_form?',
       'asset_manager_of_plants_reported_on_form?',
       'other_relationships_with_plants_reported_on_form?', 'entity_type'],
      dtype='object')

In [106]:
dfList = [utils, owners,gen_data]
[print(x.shape) for x in dfList]

(5039, 10)
(4631, 13)
(22118, 72)


[None, None, None]

In [119]:
sum([x in owners.index for x in gen_data.index])

8151